In [90]:
import numpy as np
import taichi as ti
import scipy

ti.init(arch=ti.gpu)

[Taichi] Starting on arch=cuda


In [83]:
def erf(x):
    # save the sign of x
    sign = 1 if x >= 0 else -1
    x = abs(x)

    # constants
    a1 = 0.254829592
    a2 = -0.284496736
    a3 = 1.421413741
    a4 = -1.453152027
    a5 = 1.061405429
    p = 0.3275911

    # A&S formula 7.1.26
    t = 1.0 / (1.0 + p * x)
    y = 1.0 - (((((a5 * t + a4) * t) + a3) * t + a2) * t + a1) * t * ti.exp(-x * x)
    return sign * y


def erfc(x):
    return 1 - erf(x)

In [84]:
def mgkfs_recursive_moments(T0, T1, u, mfp):
    m = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    m[0] = T0
    m[1] = T1
    for k in range(5):
        m[k + 2] = m[k + 1] * u + m[k] * (k + 1) / (2 * mfp)
    return m

In [85]:
D = 2
K = 0
b = K + D

PI = 3.1415926535897
EPS = 1e-3
CFL = 0.4

c_s = 1
T_ref = 1.0
gamma = (b + 2) / b
Rg = c_s**2 / (gamma * T_ref) 

In [106]:
rho_L, rho_R = 1.2, 1.2
u1_L, u1_R = -0.5, -0.5
u2_L, u2_R = 0.6, 0.6
T_L, T_R = 1.0, 1.0
mfp_L, mfp_R = 1.0 / (2 * Rg * T_L), 1.0 / (2 * Rg * T_R)
print(mfp_L, mfp_R)

1.0 1.0


In [107]:
T0 = erfc(-ti.sqrt(mfp_L) * u1_L) / 2.0
# T0 = scipy.special.erfc(-ti.sqrt(mfp_L) * u1_L) / 2.0
T1 = u1_L * T0 + ti.exp(-mfp_L * u1_L**2) / (2 * ti.sqrt(PI * mfp_L))
M_L = mgkfs_recursive_moments(T0, T1, u1_L, mfp_L)

T0 = erfc(ti.sqrt(mfp_R) * u1_R) / 2.0
# T0 = scipy.special.erfc(ti.sqrt(mfp_R) * u1_R) / 2.0
T1 = u1_R * T0 - ti.exp(-mfp_R * u1_R**2) / (2 * ti.sqrt(PI * mfp_R))
M_R = mgkfs_recursive_moments(T0, T1, u1_R, mfp_R)

print(M_L)
print(M_R)

[0.23974999 0.09982065 0.06996467 0.06483831 0.07252785 0.0934127
 0.13461328]
[ 0.76025001 -0.59982065  0.68003533 -0.93983831  1.48997215 -2.6246627
  5.03726172]


In [108]:
rho_i = M_L[0] * rho_L + M_R[0] * rho_R
u1 = (M_L[1] * rho_L + M_R[1] * rho_R) / rho_i
u2 = (M_L[0] * rho_L * u2_L + M_R[0] * rho_R * u2_R) / rho_i

print(rho_i)
print(u1)
print(u2)

1.2
-0.5
0.6


In [109]:
T0 = (u2_L**2 + (b - 1) * Rg * T_L) * M_L[0]
T1 = (u2_R**2 + (b - 1) * Rg * T_R) * M_R[0]
E_i = ((M_L[2] + T0) * rho_L + (M_R[2] + T1) * rho_R) / (2 * rho_i)
e_i = ti.max(E_i - (u1**2 + u2**2) / 2.0, EPS)
mfp_i = 1.0 / (2 * e_i * (gamma - 1))

print(mfp_i)

1.0000000000000002
